In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
customized_path = "/content/drive/Shareddrives/Urban Computing Project/"
sys.path.append(customized_path)

In [ ]:
%cd /content/drive/Shareddrives/Urban\ Computing\ Project/datasets

/content/drive/Shareddrives/Urban Computing Project/datasets


In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 13.7 MB/s 
     |████████████████████████████████| 15.4 MB 41 kB/s 
     |████████████████████████████████| 6.3 MB 27.5 MB/s 


In [ ]:
import pandas as pd
import fiona
import shapely
import shapely.geometry
import geopandas as gpd
from shapely import wkt

In [ ]:
latlng = pd.read_csv("/content/drive/Shareddrives/Urban Computing Project/datasets/next_stop_possibility_with_google.csv")
latlng.head()

,Unnamed: 0,Unnamed: 1,BusStop,loc,first_pos,second_pos,firstpart,secondpart,fullpart,locational_data_1,lat_1,long_1,locational_data_2,lat_2,long_2,locational_data_3,lat_3,long_3,stop_zone,googleGeoCode,googleLat,googleLon
0,2.0,3.0,MIDLAND AV/RICHMOND RD,"['MIDLAND AV', 'RICHMOND RD']",MIDLAND AV,RICHMOND RD,"MIDLAND AV,New York City,NY,United States","RICHMOND RD,New York City,NY,United States","MIDLAND AV/RICHMOND RD,New York City,NY,United...","Midland Avenue, Grant City, Staten Island, Ric...",40.577376,-74.101218,"Midland Avenue, Midland, Beaver County, Pennsy...",40.635762,-80.451319,"Richmond Road, Richmond, Staten Island, Richmo...",40.575823,-74.123824,NaN,"Richmond Rd & Midland Ave, Staten Island, NY 1...",40.582212,-74.110288
1,10.0,11.0,165 ST / 165 ST TERM,"['165 ST ', ' 165 ST TERM']",165 ST,165 ST TERM,"165 ST ,New York City,NY,United States","165 ST TERM,New York City,NY,United States","165 ST / 165 ST TERM,New York City,NY,United S...",NaN,NaN,NaN,"165 Street, Warren County, Illinois, 61417, Un...",40.764152,-90.473109,"165, Neptune Street, Port Newark/Elizabeth Mar...",40.695250,-74.135737,NaN,"Jamaica 165 St Bus Terminal, Merrick Blvd, Que...",40.706941,-73.795491
2,14.0,15.0,MC OWEN AV/BOSTON RD,"['MC OWEN AV', 'BOSTON RD']",MC OWEN AV,BOSTON RD,"MC OWEN AV,New York City,NY,United States","BOSTON RD,New York City,NY,United States","MC OWEN AV/BOSTON RD,New York City,NY,United S...",NaN,NaN,NaN,"Owen Avenue, Murray Hill, Jacksonville, Duval ...",30.312191,-81.712100,"Boston Road, New Bataan, Davao de Oro, Davao R...",7.501034,126.141372,NaN,"Mc Owen Ave & Boston Rd, Bronx, NY 10466, USA",40.889633,-73.820590
3,17.0,18.0,39 ST/2 AV,"['39 ST', '2 AV']",39 ST,2 AV,"39 ST,New York City,NY,United States","2 AV,New York City,NY,United States","39 ST/2 AV,New York City,NY,United States",NaN,NaN,NaN,"39 St, العبار, حلوان, مدينة الشارقة, الشارقة, ...",25.350127,55.433487,"2nd Avenue, Gramercy, Manhattan, New York Coun...",40.733095,-73.984396,NaN,"2nd Ave & E 39th St, New York, NY 10016, USA",40.747931,-73.973611
4,38.0,39.0,E 170 ST/GRAND CONCOURSE,"['E 170 ST', 'GRAND CONCOURSE']",E 170 ST,GRAND CONCOURSE,"E 170 ST,New York City,NY,United States","GRAND CONCOURSE,New York City,NY,United States","E 170 ST/GRAND CONCOURSE,New York City,NY,Unit...",NaN,NaN,NaN,"170 Street, Potter County, South Dakota, Unite...",44.926521,-99.725970,"Grand Concourse, The Bronx, Bronx County, New ...",40.844445,-73.911681,NaN,"E 170 St/Grand Concourse, Bronx, NY 10452, USA",40.838959,-73.914009


In [ ]:
latlng.shape

(4846, 22)

In [ ]:
latlng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         4846 non-null   float64
 1   Unnamed: 1         4846 non-null   float64
 2   BusStop            4843 non-null   object 
 3   loc                4843 non-null   object 
 4   first_pos          4843 non-null   object 
 5   second_pos         4817 non-null   object 
 6   firstpart          4846 non-null   object 
 7   secondpart         4846 non-null   object 
 8   fullpart           4846 non-null   object 
 9   locational_data_1  809 non-null    object 
 10  lat_1              809 non-null    float64
 11  long_1             809 non-null    float64
 12  locational_data_2  4520 non-null   object 
 13  lat_2              4520 non-null   float64
 14  long_2             4520 non-null   float64
 15  locational_data_3  4662 non-null   object 
 16  lat_3              4662 

In [ ]:
nyc = gpd.read_file("/content/drive/Shareddrives/Urban Computing Project/datasets/taxi_zones.shp")
nyc.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."


In [ ]:
nyc.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    263 non-null    int64   
 1   Shape_Leng  263 non-null    float64 
 2   Shape_Area  263 non-null    float64 
 3   zone        263 non-null    object  
 4   LocationID  263 non-null    int64   
 5   borough     263 non-null    object  
 6   geometry    263 non-null    geometry
dtypes: float64(2), geometry(1), int64(2), object(2)
memory usage: 14.5+ KB


In [ ]:
nyc = nyc.to_crs(epsg=4326)

In [ ]:
nyc.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [ ]:
def checkIfValidGeopoint(lat, lng):
  try:
    point = shapely.geometry.Point(lng, lat)
  except:
    return None
  for shapefile_record in nyc.iterrows():
    shape = shapely.geometry.shape(shapefile_record[1].geometry)
    if point.within(shape):
      return shapefile_record[1].zone
  return None        

In [ ]:
latlng['google_zone'] = latlng.apply(lambda x: checkIfValidGeopoint(x.googleLat, x.googleLon), axis=1)

In [ ]:
latlng.head()

,Unnamed: 0,Unnamed: 1,BusStop,loc,first_pos,second_pos,firstpart,secondpart,fullpart,locational_data_1,lat_1,long_1,locational_data_2,lat_2,long_2,locational_data_3,lat_3,long_3,stop_zone,googleGeoCode,googleLat,googleLon,google_zone
0,2.0,3.0,MIDLAND AV/RICHMOND RD,"['MIDLAND AV', 'RICHMOND RD']",MIDLAND AV,RICHMOND RD,"MIDLAND AV,New York City,NY,United States","RICHMOND RD,New York City,NY,United States","MIDLAND AV/RICHMOND RD,New York City,NY,United...","Midland Avenue, Grant City, Staten Island, Ric...",40.577376,-74.101218,"Midland Avenue, Midland, Beaver County, Pennsy...",40.635762,-80.451319,"Richmond Road, Richmond, Staten Island, Richmo...",40.575823,-74.123824,NaN,"Richmond Rd & Midland Ave, Staten Island, NY 1...",40.582212,-74.110288,Heartland Village/Todt Hill
1,10.0,11.0,165 ST / 165 ST TERM,"['165 ST ', ' 165 ST TERM']",165 ST,165 ST TERM,"165 ST ,New York City,NY,United States","165 ST TERM,New York City,NY,United States","165 ST / 165 ST TERM,New York City,NY,United S...",NaN,NaN,NaN,"165 Street, Warren County, Illinois, 61417, Un...",40.764152,-90.473109,"165, Neptune Street, Port Newark/Elizabeth Mar...",40.695250,-74.135737,NaN,"Jamaica 165 St Bus Terminal, Merrick Blvd, Que...",40.706941,-73.795491,Jamaica
2,14.0,15.0,MC OWEN AV/BOSTON RD,"['MC OWEN AV', 'BOSTON RD']",MC OWEN AV,BOSTON RD,"MC OWEN AV,New York City,NY,United States","BOSTON RD,New York City,NY,United States","MC OWEN AV/BOSTON RD,New York City,NY,United S...",NaN,NaN,NaN,"Owen Avenue, Murray Hill, Jacksonville, Duval ...",30.312191,-81.712100,"Boston Road, New Bataan, Davao de Oro, Davao R...",7.501034,126.141372,NaN,"Mc Owen Ave & Boston Rd, Bronx, NY 10466, USA",40.889633,-73.820590,Eastchester
3,17.0,18.0,39 ST/2 AV,"['39 ST', '2 AV']",39 ST,2 AV,"39 ST,New York City,NY,United States","2 AV,New York City,NY,United States","39 ST/2 AV,New York City,NY,United States",NaN,NaN,NaN,"39 St, العبار, حلوان, مدينة الشارقة, الشارقة, ...",25.350127,55.433487,"2nd Avenue, Gramercy, Manhattan, New York Coun...",40.733095,-73.984396,NaN,"2nd Ave & E 39th St, New York, NY 10016, USA",40.747931,-73.973611,UN/Turtle Bay South
4,38.0,39.0,E 170 ST/GRAND CONCOURSE,"['E 170 ST', 'GRAND CONCOURSE']",E 170 ST,GRAND CONCOURSE,"E 170 ST,New York City,NY,United States","GRAND CONCOURSE,New York City,NY,United States","E 170 ST/GRAND CONCOURSE,New York City,NY,Unit...",NaN,NaN,NaN,"170 Street, Potter County, South Dakota, Unite...",44.926521,-99.725970,"Grand Concourse, The Bronx, Bronx County, New ...",40.844445,-73.911681,NaN,"E 170 St/Grand Concourse, Bronx, NY 10452, USA",40.838959,-73.914009,West Concourse


In [ ]:
latlng = latlng.drop(columns=['valid_zone'])

In [ ]:
latlng[latlng.google_zone.isnull()].to_csv('/content/drive/Shareddrives/Urban Computing Project/datasets/missing_after_google.csv')

In [ ]:
latlng.to_csv('/content/drive/Shareddrives/Urban Computing Project/datasets/next_stop_with_zone.csv')

In [ ]:
 count = 0
for index, row in latlng.iterrows():
  #print(row['lat_2'], row['long_2'])
  count=count+checkIfValidGeopoint(row['lat_3'], row['long_3'])

print(count)  

Streaming output truncated to the last 5000 lines.
POINT (-76.19548899999999 41.240721)
POINT (-73.9035365 40.67881)
POINT (-100.0084606 32.4732663)
POINT (-90.45904899999999 41.020405)
POINT (-74.139732 40.613214)
POINT (-74.16301999999999 40.590571)
POINT (-73.7713104 40.7102283)
POINT (-90.49904599999999 40.845693)
POINT (-122.746015 49.0561156)
POINT (-73.793604 40.71047)
POINT (-94.58447009999999 39.0308466)
POINT (-73.9398247 40.7076636)
POINT (-94.565685 38.9693611)
POINT (-73.863923 40.849684)
POINT (-113.6017684 53.6248243)
POINT (-73.8671748 40.8714348)
POINT (-73.87808920000001 40.88686)
POINT (-94.59682109999999 39.0931618)
POINT (-103.708984 44.561104)
POINT (-94.584301 39.065622)
POINT (-1.64559722246905 53.5878702)
POINT (-93.33515159999999 44.7774009)
POINT (-0.1127661 51.5444338)
POINT (-92.51441800000001 43.7250115)
POINT (-90.49109229999999 14.5771087)
POINT (-73.86269 40.874654)
POINT (-0.135467 51.46615689999999)
POINT (-78.55084240000001 -0.3072253)
POINT (-74.107